In [25]:
from tensorflow.keras import models, layers, activations, initializers
import os

model = models.Sequential([
    #Camada que recebe todos os dados
    layers.Input(shape=(32,32,3)), # 28x28x1
    
    #Tratar a imagem
    layers.Resizing(32,32),
    layers.Rescaling(1./255),
    
    #Limpar da imagem
    layers.Conv2D(5, kernel_size=(3,3), strides=(1,1), padding='same'), # 10 @ 28x28
    layers.MaxPooling2D((3,3)),
    layers.Conv2D(10, kernel_size=(3,3), strides=2, padding='same'), # 10 @ 28x28
    layers.MaxPooling2D((2,2)),
    
    #Achatar 
    layers.Flatten(), # 7840
    
    #Análise da imagem
    layers.Dense(64, activation=activations.relu, kernel_initializer=initializers.RandomNormal()),
    
    layers.Dropout(0,30), #Desativa 20% dos neurônios apenas durante o treinamento
    layers.Dense(64, activation=activations.relu, kernel_initializer=initializers.RandomNormal()),
    layers.Dense(10, activation=activations.softmax, kernel_initializer=initializers.RandomNormal()) 
])

In [26]:
from tensorflow.keras import optimizers, losses, metrics

#Taxa de aprendizagem
lr = 0.005

model.compile(
    #Adam vai tentar não oscilar muito os valores
    optimizer = optimizers.Adam(
        learning_rate = lr    
    ),
    #Calcula a perda 
    loss = losses.SparseCategoricalCrossentropy(),
    #Métricas 
    metrics = [ metrics.sparse_categorical_accuracy ]
)

In [27]:
from tensorflow.keras import utils

path = './Data'
#Quantidade de dados dentro de cada batch
batch_size = 32


train = utils.image_dataset_from_directory(
    #caminho da imagem
    directory=path + '/Data/Train',
    #sim, vai embaralhar
    shuffle = True,
    #sempre embaralha a imagem da mesma forma
    seed = 1,
    subset = 'training',
    validation_split = 0.2,
    image_size = (32,32),
    #Em uma epoch, ele treina todos os batches, batches são as quantidades de repartições nos dados
    batch_size = batch_size
)

test = utils.image_dataset_from_directory(
    directory=path + '/Data/Test',
    shuffle = True,
    seed = 1,
    subset = 'validation',
    validation_split = 0.2,
    image_size = (32,32),
    batch_size = batch_size
)

Found 50000 files belonging to 10 classes.
Using 40000 files for training.
Found 10000 files belonging to 10 classes.
Using 2000 files for validation.


In [28]:
from tensorflow.keras import callbacks

#Tolerância de erro, se ele piorar 5 vezes seguidas, ele para de tentar
patience = 5
#Número de épocas
epochs = 100

model.fit(
    train,
    validation_data = test,
    epochs = epochs,
    #Nos diz seus resultados
    verbose = True,
    
    callbacks = [
        #Para que ele pare quando atingir sua paciência
        callbacks.EarlyStopping(
            monitor = 'val_loss',
            patience = patience,
            verbose = True
        )
        #Salvando o modelo
        # callbacks .ModelCheckpoint(
        #     filepath = model_path,
        #Salvando os filtros
        #     save_weights_only = False,
        #     monitor = 'loss',
        #O melhor é o que possui menor perda
        #     mode = 'min',
        #     save_best_only = True
        # )
    ]
)

Epoch 1/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - loss: 1.7178 - sparse_categorical_accuracy: 0.3613 - val_loss: 1.5327 - val_sparse_categorical_accuracy: 0.4320
Epoch 2/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 1.5414 - sparse_categorical_accuracy: 0.4378 - val_loss: 1.5296 - val_sparse_categorical_accuracy: 0.4410
Epoch 3/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 1.4700 - sparse_categorical_accuracy: 0.4667 - val_loss: 1.3992 - val_sparse_categorical_accuracy: 0.4695
Epoch 4/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - loss: 1.4178 - sparse_categorical_accuracy: 0.4871 - val_loss: 1.3960 - val_sparse_categorical_accuracy: 0.4880
Epoch 5/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 1.3943 - sparse_categorical_accuracy: 0.4988 - val_loss: 1.3656 - val_sparse_categorical_accuracy: 0.5010
Epoch 6/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - loss: 1.3695 - sparse_categorical_accuracy: 0.5104 - val_loss: 1.3774 - val_sparse_categorical_acc

In [29]:
from tensorflow.keras import models, layers, activations, initializers

model = models.Sequential([
    layers.Input(shape=(32,32,3)), 
    
    layers.Resizing(32,32),
    layers.Rescaling(1./255),
    layers.RandomRotation((-0.2, 0.2)),

    layers.Conv2D(10, kernel_size=(3,3), strides=(1,1), padding='same'),
    layers.MaxPooling2D((2,2)),
    
    layers.Flatten(),

    layers.Dense(24, activation=activations.relu, kernel_initializer=initializers.RandomNormal()),
    layers.Dropout(0.2),
    layers.Dense(24, activation=activations.relu, kernel_initializer=initializers.RandomNormal()),
    layers.Dense(10, activation=activations.softmax, kernel_initializer=initializers.RandomNormal())
])

from tensorflow.keras import optimizers, losses, metrics

lr = 0.001

model.compile(
    optimizer = optimizers.Adam(
        learning_rate = lr    
    ),
    loss = losses.SparseCategoricalCrossentropy(),
    metrics = [ metrics.sparse_categorical_accuracy ]
)

from tensorflow.keras import callbacks

patience = 5
epochs = 300
model_path = "./model.keras"

model.fit(
    train,
    validation_data = test,
    epochs = epochs,
    verbose = True,
    
    callbacks = [
        callbacks.EarlyStopping(
            monitor = 'val_loss',
            patience = patience,
            verbose = 1
        )
    ]
)

Epoch 1/300
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - loss: 2.0083 - sparse_categorical_accuracy: 0.2473 - val_loss: 1.7608 - val_sparse_categorical_accuracy: 0.3690
Epoch 2/300
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - loss: 1.8241 - sparse_categorical_accuracy: 0.3382 - val_loss: 1.6428 - val_sparse_categorical_accuracy: 0.4010
Epoch 3/300
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - loss: 1.7435 - sparse_categorical_accuracy: 0.3692 - val_loss: 1.5913 - val_sparse_categorical_accuracy: 0.4180
Epoch 4/300
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - loss: 1.6981 - sparse_categorical_accuracy: 0.3880 - val_loss: 1.6203 - val_sparse_categorical_accuracy: 0.4150
Epoch 5/300
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - loss: 1.6731 - sparse_categorical_accuracy: 0.3941 - val_loss: 1.5417 - val_sparse_categorical_accuracy: 0.4435
Epoch 6/300
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - loss: 1.6542 - sparse_categorical_accuracy: 0.4044 - val_loss: 1.5351 - val_sparse_categoric

KeyboardInterrupt: 